In [1]:
import numpy as np
import pandas as pd
import pickle
import csv
import os
import torch
from torch_geometric.data import Data

In [2]:
np.random.seed(42)
os.getcwd()

'C:\\Users\\94777\\Desktop\\covid\\foursquare'

with open('../input/yoochoose-clicks.p','rb') as f: #.p
    df = pickle.load(f)

df = pd.read_csv('input/raw/adjacency_matrix.dat', header=None)
df.columns=['session_id','timestamp','item_id','price','quantity']
df.head(5)

df.head(5)

buy_df = pd.read_csv('input/yoochoose-buys.dat', header=None)
buy_df.columns=['session_id','timestamp','item_id','price','quantity']
buy_df.head(20)

df['label'] = df.session_id.isin(buy_df.session_id)
df.head()

In [3]:
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm

class YooChooseBinaryDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseBinaryDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['adjacency_matrix.npy']
    @property
    def processed_file_names(self):
        return ['processed.dat']

    def download(self):
        pass
    
    def process(self):
        
        data_list = []
        
        for raw_path in self.raw_paths:
            # Read data from `raw_path`.
            temporal_network = np.load(raw_path).squeeze().T[:, :100,:100]
            print(temporal_network.shape)
            for network in temporal_network:
                df = pd.DataFrame(network)
                df = df.stack().reset_index()

                edge_list = np.array(df[['level_0', 'level_1' ]]).T            
                edge_index = torch.tensor(edge_list, dtype=torch.long)
                #x = torch.zeros(network.shape[0])
                y = torch.zeros(network.shape[0]) #total number of infected people
                y[0]=1
                data = Data( edge_index=edge_index, y=y) #x=x,
                data_list.append(data)
        print(np.array(data_list).shape)
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [4]:
dataset = YooChooseBinaryDataset(root='../foursquare/input')
labels = [0, 1]
dataset[1].y

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [5]:
dataset = dataset.shuffle()
train_dataset = dataset[:30]
val_dataset = dataset[30:60]
test_dataset = dataset[60:]
len(train_dataset), len(val_dataset), len(test_dataset)

(30, 30, 40)

In [6]:
from torch_geometric.data import DataLoader
batch_size= 1 #1024 ammar
train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [7]:
item_id_max = 100

In [8]:
for t_images in train_loader:
    print(t_images)

Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[2, 10000], y=[100])
Batch(batch=[100], edge_index=[

C:\Users\94777\.conda\envs\env_pytorch\lib\site-packages\torch_geometric\data\data.py:184: UserWarning: The number of nodes in your data object can only be inferred by its edge indices, and hence may result in unexpected batch-wise behavior, e.g., in case there exists isolated nodes. Please consider explicitly setting the number of nodes for this data object by assigning it to data.num_nodes.
  warnings.warn(__num_nodes_warn_msg__.format('edge'))


C:\Users\94777\.conda\envs\env_pytorch\lib\site-packages\torch_geometric\data\data.py:184: UserWarning: The number of nodes in your data object can only be inferred by its edge indices, and hence may result in unexpected batch-wise behavior, e.g., in case there exists isolated nodes. Please consider explicitly setting the number of nodes for this data object by assigning it to data.num_nodes.
  warnings.warn(__num_nodes_warn_msg__.format('edge'))

In [9]:
import torch.nn as nn
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops
class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='max') #  "Max" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()
        
    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, in_channels]

        x_j = self.lin(x_j)
        x_j = self.act(x_j)
        
        return x_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]


        new_embedding = torch.cat([aggr_out, x], dim=1)
        
        new_embedding = self.update_lin(new_embedding)
        new_embedding = self.update_act(new_embedding)
        
        return new_embedding

In [10]:
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
from pygcn.utils import load_data, accuracy
from pygcn.models import GCN

embed_dim = 128
class Net(torch.nn.Module):
    def __init__(self, num_embeddings = 1083, batch_size=10):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(embed_dim, 1)
        self.pool1 = TopKPooling(1, ratio=1)
        
        self.item_embedding = torch.nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embed_dim)
        
        self.conv2 = SAGEConv(128, 1)
        self.pool2 = TopKPooling(1, ratio=1)
        
        self.fc1 = nn.Linear(128,1)
        self.fc1_act = nn.ReLU()
        
    def forward(self, data):
        y, edge_index, batch = data.y, data.edge_index, data.batch
        y = self.item_embedding(y.type(torch.LongTensor))
        y = y.squeeze(1)        

        y = F.relu(self.conv1(y, edge_index)) 
        y, edge_index, _, batch,_, _= self.pool1(y, edge_index, None, batch)
        
        y = F.relu(self.conv2(y, edge_index))
        y, edge_index, _, batch,_, _= self.pool2(y, edge_index, None, batch)
        
        y =  self.fc1_act(self.fc1(y)).squeeze()
        #x = F.relu(self.conv2(x, edge_index))
        #x, edge_index, _, batch,_, _= self.pool2(x, edge_index, None, batch)
        ## insert
        return y , edge_index

In [11]:

device = torch.device('cpu') #ammar
model = Net(num_embeddings=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.MSELoss()

In [12]:
for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        print('input : ', data.y.shape)
        print('input_edge_index', data.edge_index.shape)
        output, edge_index = model(data)
        label = data.y.to(device)
        #loss = crit(output , label.type(torch.FloatTensor))
        print('output_edge_index :' , edge_index.shape)
        print('output : ', output.shape)
        break

input :  torch.Size([100])
input_edge_index torch.Size([2, 10000])
output_edge_index : torch.Size([2, 10000])
output :  torch.Size([100])


In [13]:
print('input edge_index :', len(data.edge_index.detach().numpy().T), data.edge_index.detach().numpy())
print('output edge_index :', len(edge_index.detach().numpy().squeeze().T), ' ---'  ,edge_index.detach().numpy().squeeze())

print('input nodes :', len(data.y.detach().numpy()), data.y.detach().numpy())
print('output nodes :', len(output.detach().numpy()), '--- '  ,output.detach().numpy())

input edge_index : 10000 [[ 0  0  0 ... 99 99 99]
 [ 0  1  2 ... 97 98 99]]
output edge_index : 10000  --- [[36 36 36 ... 99 99 99]
 [36 11  2 ... 97 98 99]]
input nodes : 100 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
output nodes : 100 ---  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [14]:
import networkx as nx

graph = nx.from_edgelist(edge_index.detach().numpy().T.squeeze())
nx.number_connected_components(graph)

1

In [16]:
def train():
    model.train()

   edge_index= 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output , label.type(torch.FloatTensor))
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 4)

In [ ]:
from sklearn.metrics import roc_auc_score
def evaluate(loader):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

    if(len(predictions) > 0):
        predictions = np.hstack(predictions)
        labels = np.hstack(labels)
        print('labels: ',labels)
        print('predictions: ',predictions)
        try:
            return roc_auc_score(labels, predictions)
        except ValueError: 
            print('roc_auc_score error')
            return 0
    else:
        return 0

In [ ]:
def run(epochs):
    train_losses = []
    train_accs =  []
    val_accs = []
    test_accs =  []
    
    for epoch in range(epochs):
        loss = train()
        train_acc = evaluate(train_loader)
        #val_acc = evaluate(val_loader)    
        #test_acc = evaluate(test_loader)

        train_losses.append(loss)
        #val_accs.append(val_acc)
        #test_accs.append(test_acc)
        
        print('Epoch: {:03d}, Loss: {:.5f}, Train Auc: {:.5f}, Val Auc: {:.5f}, Test Auc: {:.5f}'.
              format(epoch, loss, train_acc, 0, 0))# val_acc, test_acc))
    
    return train_losses, None, None #val_accs , test_accs

for data in train_loader:
    data = data.to(device)
    output = model(data)
    print(output)
    label = data.y.to(device)
    print(label)
    #loss = crit(output, label)
    break

#output.type(torch.LongTensor)
crit(output , label.type(torch.FloatTensor))

In [ ]:
train_losses,  val_accs , test_accs = run(1)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import normalize

def normal_values(train_losses):
    norm1 = np.array(train_losses) / np.linalg.norm(train_losses)
    #norm2 = normalize(np.array(train_losses)[:,np.newaxis], axis=0).ravel()
    return norm1

train_line, = plt.plot(normal_values(train_losses))
train_line.set_label('training')
#plt.plot(normal_values(val_accs))

#test_accs_line, = plt.plot(normal_values(test_accs))
#test_accs_line.set_label('test')

#validation_line, = plt.plot(normal_values(val_accs))
#validation_line.set_label('validation')
#plt.plot(normal_values(val_accs))


plt.legend()